In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
model = AutoModelForCausalLM.from_pretrained('codellama/CodeLlama-7b-Instruct-hf')
model.eval()
model.to('cuda')

/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.24s/it]
/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [3]:
model.device

device(type='cuda', index=0)

In [4]:
!nvidia-smi

Sun Aug  4 22:40:58 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:83:00.0 Off |                    0 |
| N/A   30C    P0             69W /  400W |   26773MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [5]:
tokenizer = AutoTokenizer.from_pretrained('codellama/CodeLlama-7b-Instruct-hf')

In [18]:
from transformers import pipeline

In [21]:
code_generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Generate code for an input string
input_string = "Напиши код, который сортирует список алгоритмом buble sort"
with torch.no_grad():
    generated_code = code_generator(input_string, max_length=100)[0]['generated_text']
print(generated_code)

Напиши код, который сортирует список алгоритмом buble sort.

```python
def bubble_sort(arr):
    n = len(arr)
    for i in range(n-1):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j


In [6]:
import numpy as np 
print(f'num params before SVD compression = {sum( [ np.prod(item.size())  for item in model.parameters() ])}')

num params before SVD compression = 6738546688


In [ ]:
<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n

In [16]:
system = 'Ты - гениальный программист, который помогает разработчикам писать код на языке Python. Ты всегда пишешь корректный и отформатированный код по нужному запросу'
prompts = [
    'Сгенерируй код, который считает среднее numpy-массива. В ответ выдай только программу на языке Python без пояснений',
    'Сгенерируй код, который сортирует numpy-массив. В ответ выдай только программу на языке Python  без пояснений', 
    'Сгенерируй код, который разворачивает numpy-массива. В ответ выдай только программу на языке Python  без пояснений', 
    'Сгенерируй код, который считает кумулятивное среднее спиcка. В ответ выдай только программу на языке Python  без пояснений', 
    'Сгенерируй код, который сортирует список алгоритмом buble sort. В ответ выдай только программу на языке Python  без пояснений',
    'Sort the array'
]
tokenizer.pad_token = tokenizer.eos_token

inputs = tokenizer([system + prompt for prompt in prompts], return_tensors="pt", padding=True)
model.to('cuda')
for k, v in inputs.items(): 
    inputs[k].to(model.device)
for k, v in inputs.items(): 
    print(f'{k} device = {v.device}')

with torch.no_grad():
    generation_output = model.generate(\
        input_ids = inputs['input_ids'].cuda(),\
        attention_mask = inputs['attention_mask'].cuda(),\
        #position_ids = inputs['position_ids'].cuda(),\
        use_cache = False,\
        return_dict_in_generate=True,\
 #       output_scores=False,\
        max_new_tokens = 128,\
        do_sample = True)
 #       top_p = 0.95,\
 #       repetition_penalty = 1.1,\
 #       temperature = 0.01)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


input_ids device = cpu
attention_mask device = cpu


In [17]:
tokenizer.batch_decode(generation_output.sequences)

['</s></s></s></s></s></s></s><s> Ты - гениальный программист, который помогает разработчикам писать код на языке Python. Ты всегда пишешь корректный и отформатированный код по нужному запросуСгенерируй код, который считает среднее numpy-массива. В ответ выдай только программу на языке Python без пояснений и языки нанений. языки нанесь, языки нанек нан. Языки на, Языки Язы, язек, яз и яз. Яз. Яз. яз. Яз, Яз. яз, Яз, яз, яз, яз. Язки языки."\n\\end{code}\n\\end{code}\n\\end{code}\n\\end{code}\n%}\n\\end{code.}\n\\cod.code.}\n\\cod.code.}\n\\cod.code',
 "</s></s></s></s></s><s> Ты - гениальный программист, который помогает разработчикам писать код на языке Python. Ты всегда пишешь корректный и отформатированный код по нужному запросуСгенерируй код, который сортирует numpy-массив. В ответ выдай только программу на языке Python  без пояснений. без аснении.'\n\n#\n\n#\n#\n\n#\n#\n#\n#\n#\n#\n#\n#\n#\n#\n#\n#\nПереписать ко#\n#\n# э#\n#\n#\n#\n#\n#\n#бе#\n#\n#не#\n#\n#'\n#\n#\n#'.\n#\n#\n#\n

In [16]:
model

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(151552, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-39): 40 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): SdpaAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in

In [44]:
import math
from typing import Optional, Tuple

import torch
import torch.utils.checkpoint
from torch import nn

from transformers.activations import ACT2FN
from transformers.utils import logging
from transformers import LlamaConfig

logger = logging.get_logger(__name__)

_CONFIG_FOR_DOC = "LlamaConfig"

class LlamaRMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        """
        LlamaRMSNorm is equivalent to T5LayerNorm
        """
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        variance = hidden_states.to(torch.float32).pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)

        # convert into half-precision if necessary
        if self.weight.dtype in [torch.float16, torch.bfloat16]:
            hidden_states = hidden_states.to(self.weight.dtype)

        return self.weight * hidden_states


class LlamaRotaryEmbedding(torch.nn.Module):
    def __init__(self, dim, max_position_embeddings=2048, base=10000, device=None):
        super().__init__()
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float().to(device) / dim))
        self.register_buffer("inv_freq", inv_freq)

        # Build here to make `torch.jit.trace` work.
        self.max_seq_len_cached = max_position_embeddings
        t = torch.arange(self.max_seq_len_cached, device=self.inv_freq.device, dtype=self.inv_freq.dtype)
        freqs = torch.einsum("i,j->ij", t, self.inv_freq)
        # Different from paper, but it uses a different permutation in order to obtain the same calculation
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer("cos_cached", emb.cos()[None, None, :, :], persistent=False)
        self.register_buffer("sin_cached", emb.sin()[None, None, :, :], persistent=False)

    def forward(self, x, seq_len=None):
        # x: [bs, num_attention_heads, seq_len, head_size]
        # This `if` block is unlikely to be run after we build sin/cos in `__init__`. Keep the logic here just in case.
        if seq_len > self.max_seq_len_cached:
            self.max_seq_len_cached = seq_len
            t = torch.arange(self.max_seq_len_cached, device=x.device, dtype=self.inv_freq.dtype)
            freqs = torch.einsum("i,j->ij", t, self.inv_freq)
            # Different from paper, but it uses a different permutation in order to obtain the same calculation
            emb = torch.cat((freqs, freqs), dim=-1).to(x.device)
            self.register_buffer("cos_cached", emb.cos()[None, None, :, :], persistent=False)
            self.register_buffer("sin_cached", emb.sin()[None, None, :, :], persistent=False)
        return (
            self.cos_cached[:, :, :seq_len, ...].to(dtype=x.dtype),
            self.sin_cached[:, :, :seq_len, ...].to(dtype=x.dtype),
        )


def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)


def apply_rotary_pos_emb(q, k, cos, sin, position_ids):
    gather_indices = position_ids[:, None, :, None]  # [bs, 1, seq_len, 1]
    gather_indices = gather_indices.repeat(1, cos.shape[1], 1, cos.shape[3])
    cos = torch.gather(cos.repeat(gather_indices.shape[0], 1, 1, 1), 2, gather_indices)
    sin = torch.gather(sin.repeat(gather_indices.shape[0], 1, 1, 1), 2, gather_indices)
    
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed


class SVD_LlamaMLP(nn.Module):
    def __init__(
        self,
        hidden_size: int,
        intermediate_size: int,
        hidden_act: str,
        ratio=1
    ):
        super().__init__()
        self.ratio = ratio
        low_rank = int(intermediate_size * hidden_size * self.ratio / (intermediate_size + hidden_size))
        self.gate_u_proj = nn.Linear(low_rank, intermediate_size, bias=False)
        self.gate_v_proj = nn.Linear(hidden_size, low_rank, bias=False)
        
        self.down_u_proj = nn.Linear(low_rank, hidden_size, bias=False)
        self.down_v_proj = nn.Linear(intermediate_size, low_rank, bias=False)
        
        self.up_u_proj = nn.Linear(low_rank, intermediate_size, bias=False)
        self.up_v_proj = nn.Linear(hidden_size, low_rank, bias=False)
        self.act_fn = ACT2FN[hidden_act]

    def forward(self, x):
        up = self.up_u_proj(self.up_v_proj(x))
        gate = self.gate_u_proj(self.gate_v_proj(x))
        return self.down_u_proj(self.down_v_proj(self.act_fn(gate) * up))


class SVD_LlamaAttention(nn.Module):
    """Multi-headed attention from 'Attention Is All You Need' paper"""

    def __init__(self, config: LlamaConfig, ratio=1):
        super().__init__()
        
        self.config = config
        self.hidden_size = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = self.hidden_size // self.num_heads
        self.max_position_embeddings = config.max_position_embeddings
        self.ratio = ratio # 1 means no truncate, just keep normal attn

        if (self.head_dim * self.num_heads) != self.hidden_size:
            raise ValueError(
                f"hidden_size must be divisible by num_heads (got `hidden_size`: {self.hidden_size}"
                f" and `num_heads`: {self.num_heads})."
            )
        low_rank = int(self.hidden_size * self.ratio/2)
        self.q_u_proj = nn.Linear(low_rank, self.num_heads * self.head_dim, bias=False)
        self.q_v_proj = nn.Linear(self.hidden_size, low_rank, bias=False)

        self.k_u_proj = nn.Linear(low_rank, self.num_heads * self.head_dim, bias=False)
        self.k_v_proj = nn.Linear(self.hidden_size, low_rank, bias=False)

        self.v_u_proj = nn.Linear(low_rank, self.num_heads * self.head_dim, bias=False)
        self.v_v_proj = nn.Linear(self.hidden_size, low_rank, bias=False)

        self.o_u_proj = nn.Linear(low_rank, self.hidden_size, bias=False)
        self.o_v_proj = nn.Linear(self.num_heads * self.head_dim, low_rank, bias=False)

        self.rotary_emb = LlamaRotaryEmbedding(self.head_dim, max_position_embeddings=self.max_position_embeddings)

    def _shape(self, tensor: torch.Tensor, seq_len: int, bsz: int):
        return tensor.view(bsz, seq_len, self.num_heads, self.head_dim).transpose(1, 2).contiguous()

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        output_attentions: bool = False,
        use_cache: bool = False,
        cache_position: torch.Tensor = None
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        bsz, q_len, _ = hidden_states.size()
    
        query_states = self.q_u_proj(self.q_v_proj(hidden_states)).view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)

        key_states = self.k_u_proj(self.k_v_proj(hidden_states)).view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)

        value_states = self.v_u_proj(self.v_v_proj(hidden_states)).view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)

        kv_seq_len = key_states.shape[-2]
        if past_key_value is not None:
            kv_seq_len += past_key_value[0].shape[-2]
        cos, sin = self.rotary_emb(value_states, seq_len=kv_seq_len)
 
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin, position_ids)
        # [bsz, nh, t, hd]

        if past_key_value is not None:
            # reuse k, v, self_attention
            key_states = torch.cat([past_key_value[0], key_states], dim=2)
            value_states = torch.cat([past_key_value[1], value_states], dim=2)

        past_key_value = (key_states, value_states) if use_cache else None

        attn_weights = torch.matmul(query_states, key_states.transpose(2, 3)) / math.sqrt(self.head_dim)

        if attn_weights.size() != (bsz, self.num_heads, q_len, kv_seq_len):
            raise ValueError(
                f"Attention weights should be of size {(bsz * self.num_heads, q_len, kv_seq_len)}, but is"
                f" {attn_weights.size()}"
            )

        if attention_mask is not None:
            if attention_mask.size() != (bsz, 1, q_len, kv_seq_len):
                raise ValueError(
                    f"Attention mask should be of size {(bsz, 1, q_len, kv_seq_len)}, but is {attention_mask.size()}"
                )
            attn_weights = attn_weights + attention_mask
            attn_weights = torch.max(attn_weights, torch.tensor(torch.finfo(attn_weights.dtype).min, device=attn_weights.device))

        # upcast attention to fp32
        attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(query_states.dtype)
        attn_output = torch.matmul(attn_weights, value_states)

        if attn_output.size() != (bsz, self.num_heads, q_len, self.head_dim):
            raise ValueError(
                f"`attn_output` should be of size {(bsz, self.num_heads, q_len, self.head_dim)}, but is"
                f" {attn_output.size()}"
            )

        attn_output = attn_output.transpose(1, 2)
        attn_output = attn_output.reshape(bsz, q_len, -1)

        attn_output = self.o_u_proj(self.o_v_proj(attn_output))

        if not output_attentions:
            attn_weights = None

        return attn_output, attn_weights, past_key_value

In [14]:
def find_layers(module, layers=[nn.Conv2d, nn.Linear], name=''):
    if type(module) in layers:
        return {name: module}
    res = {}
    for name1, child in module.named_children():
        res.update(find_layers(
            child, layers=layers, name=name + '.' + name1 if name != '' else name1
        ))
    return res

In [15]:
from tqdm import tqdm

In [16]:
ratio = 0.2

In [15]:
def hook(module, input, output):
    inp = input[0].detach().float()
    if inp.dim() == 2:   # for opt
        inp = inp.unsqueeze(0)
    adds = torch.matmul(inp.transpose(1,2), inp)
    adds_sum = torch.sum(adds, dim=0)
    module.raw_scaling_diag_matrix += adds_sum
    del inp, adds, adds_sum
    torch.cuda.empty_cache()

In [16]:
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        module.raw_scaling_diag_matrix = 0
        module.register_forward_hook(hook)

In [17]:
from datasets import load_dataset
def get_wikitext2(nsamples, seed, seqlen, tokenizer, dataset_cache_dir=None):
    traindata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train', cache_dir=dataset_cache_dir)
    testdata = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test', cache_dir=dataset_cache_dir)

    trainenc = tokenizer("\n\n".join(traindata['text']), return_tensors='pt')
    testenc = tokenizer("\n\n".join(testdata['text']), return_tensors='pt')

    import random
    random.seed(seed)
    trainloader = []
    for _ in range(nsamples):
        i = random.randint(0, trainenc.input_ids.shape[1] - seqlen - 1)
        j = i + seqlen
        inp = trainenc.input_ids[:, i:j]
        tar = inp.clone()
        tar[:, :-1] = -100
        trainloader.append((inp, tar))
        
    return trainloader, testenc

In [18]:
calib_loader, _ = get_wikitext2(nsamples = 256, seed = 42, seqlen = 8192, tokenizer = tokenizer, dataset_cache_dir=None)

In [19]:
len(calib_loader)

256

In [20]:
for batch in tqdm(calib_loader):
    print(f'batch len = {len(batch)}')
    #batch = {k: v for k, v in batch.items()}
    with torch.inference_mode(): 
        model(batch[0].to(model.device))

  0%|          | 0/256 [00:00<?, ?it/s]

batch len = 2


  0%|          | 1/256 [00:06<29:30,  6.94s/it]

batch len = 2


  1%|          | 2/256 [00:13<28:42,  6.78s/it]

...

 99%|█████████▉| 254/256 [28:32<00:13,  6.76s/it]

batch len = 2


100%|█████████▉| 255/256 [28:39<00:06,  6.75s/it]

batch len = 2


100%|██████████| 256/256 [28:46<00:00,  6.74s/it]


In [ ]:
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        module._forward_hooks.clear()
from tqdm import trange
for i in trange(len(layers)):
    subset = find_layers(layers[i])
    for name in subset:
        subset[name].raw_scaling_diag_matrix = subset[name].raw_scaling_diag_matrix.cpu()

In [23]:
print(1) 

1


In [17]:
dev = 'cuda'

In [30]:
profiling_mat = {}
print("Start Cholesky Decomposition...")
for i in tqdm(range(len(model.layers))):
    layer_profile = {}
    subset = find_layers(model.layers[i])
    for name in subset:
        raw_scaling_diag_matrix = subset[name].raw_scaling_diag_matrix.double().to(dev)
        try:
            scaling_diag_matrix = torch.linalg.cholesky(raw_scaling_diag_matrix)
        except Exception as e:
            print("Warning: eigen scaling_diag_matrix is not positive!")
            eigenvalues = torch.linalg.eigvalsh(raw_scaling_diag_matrix)
            raw_scaling_diag_matrix += (- eigenvalues[0] + 1e-6) * torch.eye(raw_scaling_diag_matrix.shape[0]).to(dev)
            scaling_diag_matrix = torch.linalg.cholesky(raw_scaling_diag_matrix)
            eigenvalues = None
            del eigenvalues
        layer_profile[name] = scaling_diag_matrix.cpu()
        scaling_diag_matrix = raw_scaling_diag_matrix = subset[name].raw_scaling_diag_matrix = None
        del scaling_diag_matrix, raw_scaling_diag_matrix, subset[name].raw_scaling_diag_matrix
        torch.cuda.empty_cache()
    profiling_mat[i] = layer_profile

Start Cholesky Decomposition...


  0%|          | 0/18 [00:00<?, ?it/s]

  6%|▌         | 1/18 [00:00<00:14,  1.20it/s]

 11%|█         | 2/18 [00:01<00:10,  1.52it/s]

 17%|█▋        | 3/18 [00:01<00:09,  1.66it/s]

...

 67%|██████▋   | 12/18 [00:06<00:02,  2.17it/s]

 78%|███████▊  | 14/18 [00:07<00:01,  2.12it/s]

 83%|████████▎ | 15/18 [00:07<00:01,  2.05it/s]

 89%|████████▉ | 16/18 [00:08<00:01,  1.99it/s]

100%|██████████| 18/18 [00:09<00:00,  1.97it/s]


In [32]:
import pickle 

with open('profiling_mat.pickle', 'wb') as f: 
    pickle.dump(profiling_mat, f)

In [19]:
import pickle 
with open('profiling_mat.pickle', 'rb') as f: 
    profiling_mat = pickle.load(f)

/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [20]:
model.device

device(type='cuda', index=0)

In [21]:
!nvidia-smi

Sun Aug  4 21:17:02 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:83:00.0 Off |                    0 |
| N/A   30C    P0             68W /  400W |    9985MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [45]:
from tqdm import trange
for i in trange(len(model.model.layers)):
    layer = model.model.layers[i]
    subset = find_layers(layer)

    svd_attn = SVD_LlamaAttention(config=model.config, ratio=ratio)
    svd_mlp = SVD_LlamaMLP(hidden_size=layer.hidden_size, intermediate_size=model.config.intermediate_size, hidden_act=model.config.hidden_act, ratio=ratio)

    for name in subset:
        W = subset[name].weight.data.float()
        dtype = W.dtype
        scaling_diag_matrix = profiling_mat[i][name].cuda()
        try:
            scaling_matrix_inv = torch.linalg.inv(scaling_diag_matrix)
        except Exception as e:
            print("Warning: scaling_diag_matrix is not full rank!")
            scaling_diag_matrix += 1e-6 * torch.eye(scaling_diag_matrix.shape[0]).to(dev)
            scaling_matrix_inv = torch.linalg.inv(scaling_diag_matrix)
                
        scaling_diag_matrix = scaling_diag_matrix.float()
        scaling_matrix_inv = scaling_matrix_inv.float()
        W_scale = torch.matmul(W, scaling_diag_matrix)
        
        U, S, VT = torch.linalg.svd(W_scale, full_matrices=False)
        
        num_s_after_trunc = int(W.shape[0] * W.shape[1] * ratio / (W.shape[0] + W.shape[1]))
        
        truc_s = S[:num_s_after_trunc]
        truc_u = U[:, :num_s_after_trunc]
        truc_v = torch.matmul(VT[:num_s_after_trunc, :], scaling_matrix_inv)
        truc_sigma = torch.diag(truc_s)
        
        #### Replace Attn, MLP ####
            
        sqrtSigma = torch.sqrt(truc_sigma)
        svd_u = torch.matmul(truc_u, sqrtSigma).cpu().to(dtype)
        svd_v = torch.matmul(sqrtSigma, truc_v).cpu().to(dtype)
        
        if "q_proj" in name:
            svd_attn.q_u_proj.weight.data = svd_u
            svd_attn.q_v_proj.weight.data = svd_v
        elif "k_proj" in name:
            svd_attn.k_u_proj.weight.data = svd_u
            svd_attn.k_v_proj.weight.data = svd_v
        elif "v_proj" in name:
            svd_attn.v_u_proj.weight.data = svd_u
            svd_attn.v_v_proj.weight.data = svd_v
        elif "o_proj" in name:
            svd_attn.o_u_proj.weight.data = svd_u
            svd_attn.o_v_proj.weight.data = svd_v
            layer.self_attn =  svd_attn
        elif "gate_proj" in name:
            svd_mlp.gate_u_proj.weight.data = svd_u
            svd_mlp.gate_v_proj.weight.data = svd_v
        elif "down_proj" in name:
            svd_mlp.down_u_proj.weight.data = svd_u
            svd_mlp.down_v_proj.weight.data = svd_v
        elif "up_proj" in name:
            svd_mlp.up_u_proj.weight.data = svd_u
            svd_mlp.up_v_proj.weight.data = svd_v
            layer.mlp = svd_mlp

  0%|          | 0/18 [00:00<?, ?it/s]


KeyError: 'self_attn.q_u_proj'

In [30]:
model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): SVD_LlamaAttention(
          (q_u_proj): Linear(in_features=204, out_features=2048, bias=False)
          (q_v_proj): Linear(in_features=2048, out_features=204, bias=False)
          (k_u_proj): Linear(in_features=204, out_features=2048, bias=False)
          (k_v_proj): Linear(in_features=2048, out_features=204, bias=False)
          (v_u_proj): Linear(in_features=204, out_features=2048, bias=False)
          (v_v_proj): Linear(in_features=2048, out_features=204, bias=False)
          (o_u_proj): Linear(in_features=204, out_features=2048, bias=False)
          (o_v_proj): Linear(in_features=2048, out_features=204, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): SVD_LlamaMLP(
          (gate_u_proj): Linear(in_features=364, out_features=16384, bias=False)
          (g

In [31]:
print(f'num params after SVD compression = {sum( [ np.prod(item.size())  for item in model.parameters() ])}')

num params after SVD compression = 920476672


In [32]:
920_476_672 / 2_506_172_416

0.36728385729707114

In [43]:
inputs = tokenizer("Hello, my dog is cute and ", return_tensors="pt")
model.to('cuda')
for k, v in inputs.items(): 
    inputs[k].to(model.device)
for k, v in inputs.items(): 
    print(f'{k} device = {v.device}')
generation_output = model.generate(input_ids = inputs['input_ids'].cuda(), attention_mask = inputs['attention_mask'].cuda(),use_cache = False,  return_dict_in_generate=True, output_scores=False)

input_ids device = cpu
attention_mask device = cpu


TypeError: SVD_LlamaAttention.forward() got an unexpected keyword argument 'cache_position'

In [ ]:
toke